In [1]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
from sklearn.model_selection import train_test_split
import torch
import torch.nn as nn
from torch.optim import SGD
from torch.optim import Adam
import torch.nn.functional as F
import torchvision.transforms as transforms
import torchvision.datasets as dsets

In [2]:
composed = transforms.Compose([transforms.ToTensor(), transforms.Normalize((0.5,0.5,0.5),(0.5,0.5,0.5))])
composed_ext = transforms.Compose([transforms.RandomHorizontalFlip(), 
                                   transforms.Normalize((0.5,0.5,0.5),(0.5,0.5,0.5)),
                                   transforms.RandomRotation(degrees=20),
                                   transforms.ColorJitter(brightness=0.4, contrast=0.4, saturation=0.4),
                                   transforms.ToTensor()
                                   ])
train = dsets.CIFAR10(root='./data', train=True, download=True, transform=composed_ext)
test = dsets.CIFAR10(root='./data', train=False, download=True, transform=composed)

100%|██████████| 170498071/170498071 [00:04<00:00, 39123455.47it/s]


Extracting ./data/cifar-10-python.tar.gz to ./data
Files already downloaded and verified


In [3]:
x_train = train.data
y_train = train.targets
x_test = test.data
y_test = test.targets

In [31]:
model = nn.Sequential(
    nn.Conv2d(in_channels=3, out_channels=64, kernel_size=3, stride=1, padding=1),
    nn.ReLU(),
    nn.BatchNorm2d(64),
    nn.Conv2d(in_channels=64, out_channels=64, kernel_size=3, stride=1, padding=1),
    nn.ReLU(),
    nn.BatchNorm2d(64),
    nn.MaxPool2d(kernel_size=2, stride=2),
    nn.Dropout(0.5),
    nn.Conv2d(in_channels=64, out_channels=128, kernel_size=3, stride=1, padding=1),
    nn.ReLU(),
    nn.BatchNorm2d(128),
    nn.Conv2d(in_channels=128, out_channels=128, kernel_size=3, stride=1, padding=1),
    nn.ReLU(),
    nn.BatchNorm2d(128),
    nn.MaxPool2d(kernel_size=2, stride=2),
    nn.Dropout(0.5),
    nn.Conv2d(in_channels=128, out_channels=256, kernel_size=3, stride=1, padding=1),
    nn.ReLU(),
    nn.BatchNorm2d(256),
    nn.Conv2d(in_channels=256, out_channels=256, kernel_size=3, stride=1, padding=1),
    nn.ReLU(),
    nn.BatchNorm2d(256),
    nn.MaxPool2d(kernel_size=2, stride=2),
    nn.Flatten(),
    nn.Linear(256 * 4 * 4, 4096),
    nn.BatchNorm1d(4096),
    nn.ReLU(),
    nn.Dropout(0.5),
    nn.Linear(4096, 2048),
    nn.BatchNorm1d(2048),
    nn.ReLU(),
    nn.Dropout(0.5),
    nn.Linear(2048, 1024),
    nn.BatchNorm1d(1024),
    nn.ReLU(),
    nn.Dropout(0.5),
    nn.Linear(1024, 512),
    nn.BatchNorm1d(512),
    nn.ReLU(),
    nn.Dropout(0.5),
    nn.Linear(512, 10)
)

In [32]:
optimizer = Adam(model.parameters(),lr=0.01, betas=(0.9,0.99))

loss_func = nn.CrossEntropyLoss()

def batches(input , target , batchsize = 8):
    for i in range(0,input.shape[0],batchsize):
        yield(input[i : i + batchsize],target[i : i + batchsize])

In [33]:
if torch.cuda.is_available():
    print("CUDA is available!")
else:
    print("CUDA is not available!")

device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

model = model.to(device)

CUDA is available!


In [28]:
# accuracy = 0
# num = 0
# for epoch in range(0,200):
#     accuracy = 0
#     num = 0
#     model.train()
#     for X,Y in batches(x_train,y_train,256):
#         X = torch.Tensor(X.transpose(0, 3, 1, 2)) # Convert X to a PyTorch Tensor
#         Y = torch.Tensor(Y)
#         X = X.to(device)
#         Y = Y.to(device)
#         prediction = model(X)
#         loss = loss_func(prediction , Y.long())
#         optimizer.zero_grad()
#         loss.backward()
#         optimizer.step()
#         accuracy += (prediction.max(1)[1] == Y).sum().item()
#         num += Y.size(0)
#     print(accuracy/num)

In [29]:
# accuracy = 0
# num = 0
# model.eval()
# with torch.no_grad():
#     for X,Y in batches(x_test,y_test):
#         X = torch.Tensor(X.transpose(0, 3, 1, 2)) # Convert X to a PyTorch Tensor
#         Y = torch.Tensor(Y)
#         X = X.to(device)
#         Y = Y.to(device)
#         prediction = model(X)
#         accuracy += (prediction.max(1)[1] == Y).sum().item()
#         num += Y.size(0)

#     print(accuracy/num)

In [34]:
acc_back = 0
while True:
    accuracy = 0
    num = 0
    for epoch in range(0,10):
        accuracy = 0
        num = 0
        model.train()
        for X,Y in batches(x_train,y_train,256):
            X = torch.Tensor(X.transpose(0, 3, 1, 2)) # Convert X to a PyTorch Tensor
            Y = torch.Tensor(Y)
            X = X.to(device)
            Y = Y.to(device)
            prediction = model(X)
            loss = loss_func(prediction , Y.long())
            optimizer.zero_grad()
            loss.backward()
            optimizer.step()
            accuracy += (prediction.max(1)[1] == Y).sum().item()
            num += Y.size(0)
        print(accuracy/num)

    
    accuracy = 0
    num = 0
    model.eval()
    with torch.no_grad():
        for X,Y in batches(x_test,y_test):
            X = torch.Tensor(X.transpose(0, 3, 1, 2)) # Convert X to a PyTorch Tensor
            Y = torch.Tensor(Y)
            X = X.to(device)
            Y = Y.to(device)
            prediction = model(X)
            accuracy += (prediction.max(1)[1] == Y).sum().item()
            num += Y.size(0)
        
        z = accuracy/num
        print("------------>> {} ".format(accuracy/num))
        if z < acc_back :
            break;
        else :
            acc_back = z


0.33848
0.55388
0.66094
0.72612
0.76854
0.80026
0.82418
0.84846
0.86588
0.88368
------------>> 0.8176 
0.89834
0.9117
0.92518
0.93408
0.94122
0.94604
0.95398
0.95704
0.95996
0.96332
------------>> 0.8256 
0.9637
0.96638
0.97004
0.97106
0.97288
0.9737
0.97454
0.97522
0.97642
0.97666
------------>> 0.8403 
0.97818
0.97892
0.98044
0.98106
0.98038
0.98244
0.98266
0.98232
0.9835
0.9838
------------>> 0.8427 
0.9825
0.98546
0.98482
0.98584
0.98644
0.98554
0.9863
0.98574
0.98684
0.98674
------------>> 0.8476 
0.98824
0.9876
0.98846
0.98828
0.98872
0.98826
0.98776
0.98806
0.98992
0.99002
------------>> 0.8447 
